In [1]:
DATA_DIR = '' # input('Enter file path for data files: ')

In [2]:
# Import Libraries
import random
import numpy as np
import pandas as pd

##### **Data Section 3 - GAMES**

In [3]:
# Create Games dataframes:
MDResults = pd.read_csv(DATA_DIR + 'MRegularSeasonDetailedResults.csv')
WDResults = pd.read_csv(DATA_DIR + 'WRegularSeasonDetailedResults.csv')

In [4]:
# Create Conference Abbreviations dataframes:
MTeamConferences = pd.read_csv(DATA_DIR + 'MTeamConferences.csv')
WTeamConferences = pd.read_csv(DATA_DIR + 'WTeamConferences.csv')

In [5]:
# Create Favorites dataframes (for subsequent merge and build):
MFavorites = MDResults.loc[MDResults['Season'] >= 2010, ['Season', 'DayNum', 'WTeamID', 'LTeamID', 'WLoc']].copy()
WFavorites = WDResults.loc[WDResults['Season'] >= 2010, ['Season', 'DayNum', 'WTeamID', 'LTeamID', 'WLoc']].copy()

In [6]:
# Create winning percentage statistics from (sport-reference.com) dataframes:
MWinningPCT = pd.read_csv(DATA_DIR + 'MWinningPCT.csv')

In [7]:
# Show sample output
print('Mens:', MFavorites.shape, 'Womens:', WFavorites.shape)
print('Mens:', MFavorites.columns)

Mens: (72760, 5) Womens: (70007, 5)
Mens: Index(['Season', 'DayNum', 'WTeamID', 'LTeamID', 'WLoc'], dtype='object')


In [8]:
# Create new columns (Mens) based on the value of the 'WLoc' column
MFavorites['WLocH'] = MFavorites['WLoc'].apply(lambda x: 1 if x == 'H' else 0)
MFavorites['WLocA'] = MFavorites['WLoc'].apply(lambda x: 1 if x == 'A' else 0)
MFavorites['WLocG'] = 1
MFavorites['LLocH'] = MFavorites['WLoc'].apply(lambda x: 0 if x == 'H' else 1)
MFavorites['LLocA'] = MFavorites['WLoc'].apply(lambda x: 0 if x == 'A' else 1)
MFavorites['LLocG'] = 1

# Create new columns (Womens) based on the value of the 'WLoc' column
WFavorites['WLocH'] = WFavorites['WLoc'].apply(lambda x: 1 if x == 'H' else 0)
WFavorites['WLocA'] = WFavorites['WLoc'].apply(lambda x: 1 if x == 'A' else 0)
WFavorites['WLocG'] = 1
WFavorites['LLocH'] = WFavorites['WLoc'].apply(lambda x: 0 if x == 'H' else 1)
WFavorites['LLocA'] = WFavorites['WLoc'].apply(lambda x: 0 if x == 'A' else 1)
WFavorites['LLocG'] = 1

# Show sample output
print('Mens:', MFavorites.shape, 'Womens:', WFavorites.shape)
print('Mens:', MFavorites.columns)
MFavorites.tail(3)

Mens: (72760, 11) Womens: (70007, 11)
Mens: Index(['Season', 'DayNum', 'WTeamID', 'LTeamID', 'WLoc', 'WLocH', 'WLocA',
       'WLocG', 'LLocH', 'LLocA', 'LLocG'],
      dtype='object')


,Season,DayNum,WTeamID,LTeamID,WLoc,WLocH,WLocA,WLocG,LLocH,LLocA,LLocG
106831,2023,113,1429,1461,A,0,1,1,1,0,1
106832,2023,113,1433,1386,A,0,1,1,1,0,1
106833,2023,113,1437,1462,A,0,1,1,1,0,1


In [9]:
# Merge 'Favorites' with 'Team Conferences' dataframes on 'Season' and 'WTeamID' columns
MFavorites_df = pd.merge(MFavorites, MTeamConferences, how='left', left_on=['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])
WFavorites_df = pd.merge(WFavorites, WTeamConferences, how='left', left_on=['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])

# Rename, reorder, and drop unwanted columns from merged dataframe
order = [0, 1, 2, 3, 7, 4, 5, 6, 10, 8, 9, 11, 12]
rename = {'ConfAbbrev':'WConf'}

MFavorites_df = MFavorites_df.iloc[:,order].drop(columns=['WLoc', 'TeamID'])
WFavorites_df = WFavorites_df.iloc[:,order].drop(columns=['WLoc', 'TeamID'])
MFavorites_df = MFavorites_df.rename(columns=rename)
WFavorites_df = WFavorites_df.rename(columns=rename)

# Perform second merge this time on 'Season' and 'LTeamID' columns
MFavorites2_df = pd.merge(MFavorites_df, MTeamConferences, how='left', left_on=['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])
WFavorites2_df = pd.merge(WFavorites_df, WTeamConferences, how='left', left_on=['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])

# Rename and drop unwanted and columns from merged dataframe
rename = {'ConfAbbrev':'LConf'}

MFavorites2_df = MFavorites2_df.drop(columns=['TeamID'])
WFavorites2_df = WFavorites2_df.drop(columns=['TeamID'])
MFavorites2_df = MFavorites2_df.rename(columns=rename)
WFavorites2_df = WFavorites2_df.rename(columns=rename)

# Create a new column based on the equality of the 'WConf' and 'LConf' columns
MFavorites2_df['WLocC'] = MFavorites2_df.apply(lambda x: 1 if x['WConf'] == x['LConf'] else 0, axis=1)
MFavorites2_df['LLocC'] = MFavorites2_df.apply(lambda x: 1 if x['WConf'] == x['LConf'] else 0, axis=1)
WFavorites2_df['WLocC'] = WFavorites2_df.apply(lambda x: 1 if x['WConf'] == x['LConf'] else 0, axis=1)
WFavorites2_df['LLocC'] = WFavorites2_df.apply(lambda x: 1 if x['WConf'] == x['LConf'] else 0, axis=1)

# Reorder and drop unwanted columns from merged dataframe
order = [0, 1, 2, 3, 4, 12, 5, 6, 7, 13, 8, 9, 10, 11]
MFavorites2_df = MFavorites2_df.iloc[:,order].drop(columns=['WConf', 'LConf'])
WFavorites2_df = WFavorites2_df.iloc[:,order].drop(columns=['WConf', 'LConf'])

# Show sample output
print('Mens:', MFavorites2_df.shape, 'Womens:', WFavorites2_df.shape)
MFavorites2_df.head(3)

Mens: (72760, 12) Womens: (70007, 12)


,Season,DayNum,WTeamID,LTeamID,WLocG,WLocC,WLocH,WLocA,LLocG,LLocC,LLocH,LLocA
0,2010,7,1143,1293,1,0,1,0,1,0,0,1
1,2010,7,1314,1198,1,0,1,0,1,0,0,1
2,2010,7,1326,1108,1,0,1,0,1,0,0,1


In [10]:
# Merge 'Favorites' with 'Team Conferences' dataframes on 'Season' and 'WTeamID' columns
MFavorites3_df = pd.merge(MFavorites2_df, MWinningPCT, how='left', left_on=['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])
#WFavorites3_df = pd.merge(WFavorites2_df, WWinningPCT, how='left', left_on=['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])

# Rename, reorder, and drop unwanted columns from merged dataframe
order = [0, 1, 2, 3, 4, 13, 5, 14, 6, 15, 7, 16, 8, 9, 10, 11, 12]
rename = {'WinPCT':'WWinPCT', 'ConfPCT':'WConfPCT', 'HomePCT':'WHomePCT', 'AwayPCT':'WAwayPCT'}

MFavorites3_df = MFavorites3_df.iloc[:,order].drop(columns=['TeamID'])
#WFavorites3_df = WFavorites3_df.iloc[:,order].drop(columns=['TeamID'])
MFavorites3_df = MFavorites3_df.rename(columns=rename)
#WFavorites3_df = WFavorites3_df.rename(columns=rename)

# Show sample output
print('Mens:', MFavorites3_df.shape)#, 'Womens:', WFavorites3_df.shape)
MFavorites3_df.head(3)

Mens: (72760, 16)


,Season,DayNum,WTeamID,LTeamID,WLocG,WWinPCT,WLocC,WConfPCT,WLocH,WHomePCT,WLocA,WAwayPCT,LLocG,LLocC,LLocH,LLocA
0,2010,7,1143,1293,1,0.685714,0,0.722222,1,0.937500,0,0.500000,1,0,0,1
1,2010,7,1314,1198,1,0.540541,0,0.312500,1,0.736842,0,0.333333,1,0,0,1
2,2010,7,1326,1108,1,0.783784,0,0.777778,1,0.944444,0,0.545455,1,0,0,1


In [11]:
# Perform second merge this time on 'Season' and 'LTeamID' columns
MFavorites4_df = pd.merge(MFavorites3_df, MWinningPCT, how='left', left_on=['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])
#WFavorites4_df = pd.merge(WFavorites3_df, WWinningPCT, how='left', left_on=['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])

# Rename, reorder, and drop unwanted columns from merged dataframe
order = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 17, 13, 18, 14, 19, 15, 20, 16]
rename = {'WinPCT':'LWinPCT', 'ConfPCT':'LConfPCT', 'HomePCT':'LHomePCT', 'AwayPCT':'LAwayPCT'}

MFavorites4_df = MFavorites4_df.iloc[:,order].drop(columns=['TeamID'])
#WFavorites4_df = WFavorites4_df.iloc[:,order].drop(columns=['TeamID'])
MFavorites4_df = MFavorites4_df.rename(columns=rename)
#WFavorites4_df = WFavorites4_df.rename(columns=rename)

# Show sample output
print('Mens:', MFavorites4_df.shape)#, 'Womens:', WFavorites3_df.shape)
MFavorites4_df.head(3)

Mens: (72760, 20)


,Season,DayNum,WTeamID,LTeamID,WLocG,WWinPCT,WLocC,WConfPCT,WLocH,WHomePCT,WLocA,WAwayPCT,LLocG,LWinPCT,LLocC,LConfPCT,LLocH,LHomePCT,LLocA,LAwayPCT
0,2010,7,1143,1293,1,0.685714,0,0.722222,1,0.937500,0,0.500000,1,0.861111,0,0.944444,0,1.000000,1,0.733333
1,2010,7,1314,1198,1,0.540541,0,0.312500,1,0.736842,0,0.333333,1,0.218750,0,0.222222,0,0.307692,1,0.166667
2,2010,7,1326,1108,1,0.783784,0,0.777778,1,0.944444,0,0.545455,1,0.064516,0,0.111111,0,0.200000,1,0.000000


In [12]:
# Create four new columns (Mens) to calculate the winning percentages and determine if winning team was the favorite
mwloc_sum = MFavorites4_df['WLocG'] + MFavorites4_df['WLocC'] + MFavorites4_df['WLocH'] + MFavorites4_df['WLocA']
MFavorites4_df['WTPCT'] = ((MFavorites4_df['WLocG'] * MFavorites4_df['WWinPCT']) + (MFavorites4_df['WLocC'] * MFavorites4_df['WConfPCT']) +
               (MFavorites4_df['WLocH'] * MFavorites4_df['WHomePCT']) + (MFavorites4_df['WLocA'] * MFavorites4_df['WAwayPCT'])) / mwloc_sum

mlloc_sum = MFavorites4_df['LLocG'] + MFavorites4_df['LLocC'] + MFavorites4_df['LLocH'] + MFavorites4_df['LLocA']
MFavorites4_df['LTPCT'] = ((MFavorites4_df['LLocG'] * MFavorites4_df['LWinPCT']) + (MFavorites4_df['LLocC'] * MFavorites4_df['LConfPCT']) +
               (MFavorites4_df['LLocH'] * MFavorites4_df['LHomePCT']) + (MFavorites4_df['LLocA'] * MFavorites4_df['LAwayPCT'])) / mlloc_sum

MFavorites4_df['WFAV'] = (MFavorites4_df['WTPCT'] - MFavorites4_df['LTPCT']).ge(0).astype(int)
MFavorites4_df['LFAV'] = (MFavorites4_df['WTPCT'] - MFavorites4_df['LTPCT']).lt(0).astype(int)

In [13]:
# Show sample output
print('Mens:', MFavorites4_df.shape)#, 'Womens:', WFavorites4_df.shape)
MFavorites4_df.head(3)

Mens: (72760, 24)


,Season,DayNum,WTeamID,LTeamID,WLocG,WWinPCT,WLocC,WConfPCT,WLocH,WHomePCT,...,LLocC,LConfPCT,LLocH,LHomePCT,LLocA,LAwayPCT,WTPCT,LTPCT,WFAV,LFAV
0,2010,7,1143,1293,1,0.685714,0,0.722222,1,0.937500,...,0,0.944444,0,1.000000,1,0.733333,0.811607,0.797222,1,0
1,2010,7,1314,1198,1,0.540541,0,0.312500,1,0.736842,...,0,0.222222,0,0.307692,1,0.166667,0.638691,0.192708,1,0
2,2010,7,1326,1108,1,0.783784,0,0.777778,1,0.944444,...,0,0.111111,0,0.200000,1,0.000000,0.864114,0.032258,1,0


In [14]:
# Bring back in original 'Games' dataframe (for subsequent merge and build):
MFavorites5_df = MDResults.loc[MDResults['Season'] >= 2010].copy()
WFavorites5_df = WDResults.loc[WDResults['Season'] >= 2010].copy()

# Merge 'Favorites' with original 'Games' dataframes on 'Season' and 'WTeamID' columns
MFavorites6_df = pd.merge(MFavorites4_df, MFavorites5_df, how='left', left_on=['Season', 'DayNum', 'WTeamID', 'LTeamID'], right_on = ['Season', 'DayNum', 'WTeamID', 'LTeamID'])
#WFavorites6_df = pd.merge(WFavorites4_df, WFavorites5_df, how='left', left_on=['Season', 'DayNum', 'WTeamID', 'LTeamID'], right_on = ['Season', 'DayNum', 'WTeamID', 'LTeamID'])



In [15]:
# Show sample output
print('Mens:', MFavorites6_df.shape)#, 'Womens:', WFavorites6_df.shape)
MFavorites6_df.head(3)

Mens: (72760, 54)


,Season,DayNum,WTeamID,LTeamID,WLocG,WWinPCT,WLocC,WConfPCT,WLocH,WHomePCT,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,7,1143,1293,1,0.685714,0,0.722222,1,0.937500,...,21,10,15,11,20,11,17,7,3,24
1,2010,7,1314,1198,1,0.540541,0,0.312500,1,0.736842,...,23,14,17,13,16,15,20,14,2,18
2,2010,7,1326,1108,1,0.783784,0,0.777778,1,0.944444,...,17,11,20,11,24,7,13,4,5,16


In [16]:
# Split merged 'Favorites' (version 6) dataframes (for subsequent build and matchups):

MFavorites7_df = MFavorites6_df.loc[:, ['Season', 'DayNum', 'WTeamID', 'LTeamID', 'WTPCT', 'LTPCT', 'WFAV', 'LFAV', 'WScore', 'LScore', 
                                        'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 
                                        'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 
                                        'LTO', 'LStl', 'LBlk', 'LPF']].copy()

split_win = MFavorites7_df.loc[:, ['Season', 'DayNum', 'WTeamID', 'WTPCT', 'WFAV', 'WScore', 'LScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 
                                   'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']].copy()


split_loss = MFavorites7_df.loc[:, ['Season', 'DayNum', 'LTeamID', 'LTPCT', 'LFAV', 'WScore', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 
                                    'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']].copy()

# Rename columns to prepare for concatenate dataframes
rename_win = {'WTeamID':'TeamID', 'WTPCT':'TPCT', 'WFAV':'WIN', 'WFGM':'FGM', 'WFGA':'FGA', 'WFGM3':'FGM3', 'WFGA3':'FGA3', 
              'WFTM':'FTM', 'WFTA':'FTA', 'WOR':'OR', 'WDR':'DR', 'WAst':'AST', 'WTO':'TOV', 'WStl':'STL', 'WBlk':'BLK', 'WPF':'PF'}

rename_loss = {'LTeamID':'TeamID', 'LTPCT':'TPCT', 'LFAV':'WIN', 'LFGM':'FGM', 'LFGA':'FGA', 'LFGM3':'FGM3', 'LFGA3':'FGA3', 
              'LFTM':'FTM', 'LFTA':'FTA', 'LOR':'OR', 'LDR':'DR', 'LAst':'AST', 'LTO':'TOV', 'LStl':'STL', 'LBlk':'BLK', 'LPF':'PF'}

split_win = split_win.rename(columns=rename_win)
split_loss = split_loss.rename(columns=rename_loss)

# Concatenate the two dataframes
MFavorites8_df = pd.concat([split_win, split_loss])
MFavorites8_df = MFavorites8_df.sort_values(by=['Season', 'DayNum', 'TeamID'])

In [17]:
# Show sample output
print('Mens:', MFavorites8_df.shape)#, 'Womens:', WFavorites8_df.shape)

# select rows where 'Season' is 2021, 'WTeamID' is 456, and 'LTeamID' is 123
MFavorites8_df.loc[(MFavorites8_df['Season'] == 2013) & (MFavorites8_df['DayNum'] == 73) & ((MFavorites8_df['TeamID'] == 1331) | (MFavorites8_df['TeamID'] == 1394)), :]

Mens: (145520, 20)


,Season,DayNum,TeamID,TPCT,WIN,WScore,LScore,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,AST,TOV,STL,BLK,PF
18514,2013,73,1331,0.607688,1,64,52,18,41,5,19,23,28,8,27,10,15,3,1,16
18514,2013,73,1394,0.264122,0,64,52,19,53,3,16,11,15,11,16,8,13,8,0,23


In [18]:
print('Missing Values:', MFavorites8_df.isna().sum().sum())
#print('Missing Values:', WFavorites8_df.isna().sum().sum())

Missing Values: 617


In [19]:
# Drop rows where total winning percentage is missing, but exclude the 2023 Season
mens_dropped = MFavorites8_df.loc[MFavorites8_df.TPCT.notnull(), :].copy()
#womens_dropped = WFavorites8_df.loc[WFavorites8_df.TPCT.notnull(), :].copy()

print('Missing Values:', mens_dropped.isna().sum().sum())
#print('Missing Values:', womens_dropped.isna().sum().sum())

Missing Values: 0


In [20]:
# Combine related columns
mens_dropped['FGPCT'] = mens_dropped['FGM'] / mens_dropped['FGA']
mens_dropped['FG3PCT'] = mens_dropped['FGM3'] / mens_dropped['FGA3']
mens_dropped['FTPCT'] = mens_dropped['FTM'] / mens_dropped['FTA']
mens_dropped['Luck'] = [random.uniform(0, 1) for _ in range(len(mens_dropped))]

#womens_dropped['FGPCT'] = womens_dropped['FGM'] / womens_dropped['FGA']
#womens_dropped['FG3PCT'] = womens_dropped['FGM3'] / womens_dropped['FGA3']
#womens_dropped['FTPCT'] = womens_dropped['FTM'] / womens_dropped['FTA']
#womens_dropped['Luck'] = [random.uniform(0, 1) for _ in range(len(womens_dropped))]

# Reorder and drop unwanted columns from merged dataframe
order = [0, 1, 2, 3, 23, 5, 6, 20, 7, 8, 21, 9, 10, 22, 11, 12, 13, 14, 15, 16, 17, 18, 19, 4]

mens_dropped = mens_dropped.iloc[:,order].drop(columns=['FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA'])
#womens_dropped = womens_dropped.iloc[:,order].drop(columns=['FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA'])

# Show sample output
print('Mens:', mens_dropped.shape)#, 'Womens:', womens_dropped.shape)
mens_dropped.head(3)

Mens: (144903, 18)


,Season,DayNum,TeamID,TPCT,Luck,WScore,LScore,FGPCT,FG3PCT,FTPCT,OR,DR,AST,TOV,STL,BLK,PF,WIN
3,2010,7,1107,0.192708,0.638798,75,43,0.272727,0.178571,0.571429,12,27,10,32,12,4,27,0
2,2010,7,1108,0.032258,0.849208,100,60,0.344262,0.411765,0.550000,11,24,7,13,4,5,16,0
0,2010,7,1143,0.811607,0.663655,75,70,0.461538,0.416667,0.687500,13,19,15,12,8,1,14,1


In [21]:
# Normalize selected columns using min-max range
def normalize_minmax(column):
    """
    Normalize a column to a min-max range.
    """
    min_val = min(column)
    max_val = max(column)
    normalized_col = [(x - min_val) / (max_val - min_val) for x in column]
    return normalized_col

# Select the columns you want to normalize
selected_cols = ['OR', 'DR', 'AST', 'TOV', 'STL', 'BLK', 'PF']

# Normalize the selected columns in your dataframe
for col in selected_cols:
    mens_dropped[col] = normalize_minmax(mens_dropped[col])

# Show sample output
print('Mens:', mens_dropped.shape)#, 'Womens:', womens_dropped.shape)
mens_dropped.head(3)

Mens: (144903, 18)


,Season,DayNum,TeamID,TPCT,Luck,WScore,LScore,FGPCT,FG3PCT,FTPCT,OR,DR,AST,TOV,STL,BLK,PF,WIN
3,2010,7,1107,0.192708,0.638798,75,43,0.272727,0.178571,0.571429,0.315789,0.46,0.243902,0.794872,0.461538,0.222222,0.613636,0
2,2010,7,1108,0.032258,0.849208,100,60,0.344262,0.411765,0.550000,0.289474,0.40,0.170732,0.307692,0.153846,0.277778,0.363636,0
0,2010,7,1143,0.811607,0.663655,75,70,0.461538,0.416667,0.687500,0.342105,0.30,0.365854,0.282051,0.307692,0.055556,0.318182,1


In [22]:
# Standardize selected columns using standard scaler
def standardize_scaler(column):
    """
    Standardize a column using standard scaler.
    """
    mean_val = sum(column) / len(column)
    std_val = (sum([(x - mean_val)**2 for x in column]) / (len(column)-1))**0.5
    standardized_col = [(x - mean_val) / std_val for x in column]
    return standardized_col

# Select the columns you want to standardize
selected_cols = ['WScore', 'LScore']

# Standardize the selected columns in your dataframe
for col in selected_cols:
    mens_dropped[col] = standardize_scaler(mens_dropped[col])
    
# Show sample output
print('Mens:', mens_dropped.shape)#, 'Womens:', womens_dropped.shape)
mens_dropped.head(3)

Mens: (144903, 18)


,Season,DayNum,TeamID,TPCT,Luck,WScore,LScore,FGPCT,FG3PCT,FTPCT,OR,DR,AST,TOV,STL,BLK,PF,WIN
3,2010,7,1107,0.192708,0.638798,-0.086512,-1.946539,0.272727,0.178571,0.571429,0.315789,0.46,0.243902,0.794872,0.461538,0.222222,0.613636,0
2,2010,7,1108,0.032258,0.849208,2.203883,-0.371905,0.344262,0.411765,0.550000,0.289474,0.40,0.170732,0.307692,0.153846,0.277778,0.363636,0
0,2010,7,1143,0.811607,0.663655,-0.086512,0.554350,0.461538,0.416667,0.687500,0.342105,0.30,0.365854,0.282051,0.307692,0.055556,0.318182,1


In [23]:
games_mens = mens_dropped.to_csv('games-mens.csv',index=False)
#games_womens = womens_dropped.to_csv('games-womens.csv',index=False)